# Analyse and plot output from OpenIFS

Files are named EXPID_FREQ_STARTDATE_ENDDATE_GRID_LEVEL.nc 
* FREQ can be "1d" (daily), "1m" (monthly), etc. 
* GRID can be "regular" (regular grid where lon, lat are 1D) or "reduced" (original reduced Gaussian grid).
* LEVEL can be "sfc" (surface variables, e.g. 2m temp, precipitation), "pl" (pressure levels), "pv" (potential vorticity levels)

See ECMWF parameter database for variable names and descriptions. 
https://apps.ecmwf.int/codes/grib/param-db

In [ ]:
# Load necessary modules
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cmocean 
import os

## Where to find data

In [ ]:
# Find esm-experiments in your WORK dir
esmdir = '%s/esm-experiments/' % (os.environ['WORK'],)

## Read OpenIFS data

In [2]:
# Frequency to read in 
freq = '1d'
# Which runs to read
expnames = ['OIFS-KJK001']
# Which time periods to read in
times = [slice('1979-01-01','1979-01-05')]
# Which levels to read for each run
levels = ['sfc','pl']

In [ ]:
# Put data from all runs in one list
ds_all = []

for exp,time,lev in zip(expnames, times, levels):
    
    # Find all relevant files with wildcards
    files = '%s/%s/outdata/oifs/*%s*regular_%s.nc' % (esmdir,exp,freq,lev)
    
    # Open multi-file data set. We need to use cftime since the normal python calendar stops working after 2300. 
    # We only read time set in the times list above
    # Also, we rename time variable from time_counter to time to make life easier
    # Optimized reading from Sebastian Wahl
    _ds = xr.open_mfdataset(files,combine='nested', 
                            concat_dim="time_counter", use_cftime=True,
                            data_vars='minimal', coords='minimal', compat='override',
                            parallel=True).rename({'time_counter':'time'}).sel(time=time)
    ds_all.append(_ds)

## Plot mean T2m 

In [ ]:
mean_list = []

for i,_ds in enumerate(ds_all):
    
    # Select T2m
    _mean = _ds['2t'].mean('time').compute()
    # Add experiment as coordinate
    _mean.coords['exp'] = expnames[i]
    
    mean_list.append(_mean)
    
ds_mean = xr.concat( mean_list, dim='exp' )

In [ ]:
ds_mean.plot(x='lon',y='lat',col='exp',col_wrap=2,
             cmap=cmocean.cm.thermal,
             cbar_kwargs={'label': 'SST [K]'})

## Plot surface solar radiation and precip over Honolulu

In [ ]:
flux_list = []

for i, _ds in enumerate(ds_all):
    
    # Select solar rad
    # Logic is SSR = Surface Solar Radiation
    # Similarly, TTR = Top of atmosphere Thermal Radiation
    # OpenIFS grid has lon=[0,360]
    _sw = _ds['ssr'].sel(lat=21.3,method='nearest').sel(lon=202.1,method='nearest')
    
    # Select precip
    _tp = _ds['tp'].sel(lat=21.3,method='nearest').sel(lon=202.1,method='nearest')
    
    # Put into one dataset
    _ds = xr.merge( [_sw, _tp])
    _ds.coords['exp'] = expnames[i]
    
    flux_list.append(_ds)
    
ds_flux = xr.concat( flux_list, dim='exp' )

In [ ]:
fig, ax = plt.subplots(1,2)
axs = ax.flat

# Precip is in m/s
# Scale to mm/day
pscale = 1000 * 86400

ds_flux['ssr'].plot(hue='exp',ax=axs[0])
(ds_flux['tp'] * pscale).plot(hue='exp',ax=axs[1])

## Plot jet stream over North Atlantic

In [ ]:
jet_list = []

for i, _ds in enumerate(ds_all):
    
    # Select zonal wind
    # over North Atlantic (60W-0E, 20-70N)
    # take zonal and time mean
    _ds['u'].sel(lon=slice(300,360)).sel(lat=slice(20,70)).mean('lon').mean('time')
    _ds.coords['exp'] = expnames[i]
    
    jet_list.append(_ds)
    
ds_jet = xr.concat(jet_list, dim='exp')

In [ ]:
ds_mean.plot(x='lat',y='pressure_levels',col='exp',col_wrap=2,
             cmap=cmocean.cm.curl,
             cbar_kwargs={'label': 'U [m/s]'})